# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Буланбаев Артур Ильдарович`

Группа: `MADE-DS-11`

In [ ]:
# install required packages
!pip install geopandas
!pip install OSMPythonTools
!pip install pycountry
!sudo apt install python3-rtree
!pip install "rtree>=0.8,<0.9"
!pip install openrouteservice

In [ ]:
import os
import json
import pyproj
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

# packages for processing geodata
import folium
import geopandas as gpd
from shapely import geometry, wkt
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
import pycountry
import rtree
from openrouteservice import client

import warnings
warnings.filterwarnings('ignore')

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

Loading world map and polygon of Afghanistan

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world_bounds = world.geometry.bounds
minx, maxx = world_bounds.minx.min(), world_bounds.maxx.max()
miny, maxy = world_bounds.miny.min(), world_bounds.maxy.max()

afg_poly = world.loc[world['name'] == 'Afghanistan'].geometry.values[0]

The algorithm starts from random point on the world map. Next point is generated randomly on the circle with radius equal to 1.5 distance from nearest current point (of all generated by now) to Afghanistan.

`points` - all generated points

`better_points` - only successful points which were closer to Afghanistan then the previous ones

`routes` - all trajectories that include "bad" points (for full visualization)

In [ ]:
def search_afghanistan():
    point = nearest_point = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
    points = [point]
    better_points = [point]
    routes = []
    min_dist = afg_poly.distance(point)
    while afg_poly.contains(point) == False:
        dist = afg_poly.distance(point)
        if dist < min_dist:
            min_dist = dist
            nearest_point = point
            better_points.append(point)
        else:
            routes.append(better_points + [point])
        R = 1.5 * min_dist
        phi = np.random.uniform(0, 2 * np.pi)
        point = Point(
            np.clip(nearest_point.x + R * np.cos(phi), 0.9 * minx, 0.9 * maxx),
            np.clip(nearest_point.y + R * np.sin(phi), 0.9 * miny, 0.9 * maxy)
        )
        points.append(point)
    better_points.append(point)
    print('Number of points:', len(points))
    print('Number of successful guesses:', len(better_points))
    return points, better_points, routes

Visualization contains the borders of Afghanistan, the points are consequently connected. 

`failures` option allows to see only successful guesses (False), or full set of points with bad guesses (True).

In [ ]:
def visualize_search(failures=True):
    # map
    m = folium.Map(
        location=[points[0].y, points[0].x],
        tiles='OpenStreetMap',
        zoom_start=3
    )
    
    # starting point
    folium.Marker(
        location=[points[0].y, points[0].x],
        popup='Starting point',
        icon=folium.Icon(color='blue', icon='play')
    ).add_to(m)
    
    # country border
    folium.GeoJson(
        afg_poly,
        style_function=lambda x: {'color': 'black'},
        name='Afghanistan'
    ).add_to(m)
    
    # line of points
    folium.GeoJson(
        LineString(better_points),
        name='points'
    ).add_to(m)
    
    # markered points
    for point in points[1:]:
        if point in better_points:
            folium.Marker(
                location=[point.y, point.x],
                popup='Good guess',
                icon=folium.Icon(color='green', icon='check')
            ).add_to(m)
            
    if failures:
        # bad guesses
        for route in routes:
            folium.GeoJson(
                LineString(route),
                name='route'
            ).add_to(m)
        
        # markered bad points
        for point in points[1:]:
            if point not in better_points:
                folium.Marker(
                    location=[point.y, point.x],
                    popup='Bad guess',
                    icon=folium.Icon(color='red', icon='backward')
                ).add_to(m)
        
    return m

In [ ]:
points, better_points, routes = search_afghanistan()

In [ ]:
search = visualize_search(failures=False)
search_full = visualize_search(failures=True)

In [ ]:
search

In [ ]:
search_full

In [ ]:
# search.save('afghanistan_search.html')
# search_full.save('afghanistan_search_full.html')

In [ ]:
afg_point = points[-1]

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
nominatim = Nominatim()
overpass = Overpass()

Load id's of countries, leave those which id ends with '0'.

In [ ]:
%%time
country_id = {}
for country in pycountry.countries:
    areaid = nominatim.query(country.name).areaId()
    if str(areaid).endswith('0'):
        country_id[country.name] = areaid

In [ ]:
country_id

Let us find the country with the biggest amount of residential buildings.

In [ ]:
country_max_residential = ''
max_residential = 0
for country in tqdm(country_id.keys()):
    query = overpassQueryBuilder(area=country_id[country], 
                              elementType=['node', 'way'], 
                              selector=['"building"~"house|residential|apartments|detached|terrace|hut"'], 
                              out='count')
    result = overpass.query(query, timeout=180)
    residential = result.countElements()
    if residential > max_residential:
        max_residential = residential
        country_max_residential = country

Let's observe the country name and derive ATMs.

In [ ]:
country_max_residential

In [ ]:
query = overpassQueryBuilder(area=country_id[country_max_residential],
                             elementType='node',
                             selector=['"amenity"="atm"'], 
                             out='body')
result = overpass.query(query, timeout=60)

In [ ]:
# number of ATMs found
result.countElements()

Find 5 nearest among them using Rtree.

In [ ]:
idx = rtree.index.Rtree()
for i, point in enumerate(result.elements()):
    idx.insert(i, (point.lat(), point.lon()))

nearest_atm = list(idx.nearest((afg_point.y, afg_point.x), 5))

In [ ]:
brazil_poly = world.loc[world['name'] == 'Brazil'].geometry.values[0]

def visualize_ATM():
    # map
    m = folium.Map(
        location=[afg_point.y, afg_point.x],
        tiles='OpenStreetMap',
        zoom_start=3
    )

    # afg point
    folium.Marker(
        location=[afg_point.y, afg_point.x],
        popup='Point in Afghanistan',
        icon=folium.Icon()
    ).add_to(m)

    # nearest ATM
    for i in nearest_atm:
        folium.Marker(
            location=[result.elements()[i].lat(), result.elements()[i].lon()],
            popup='Top 5 nearest ATM',
            icon=folium.Icon(color='green', icon='check')
        ).add_to(m)

    # some far ATM
    for i in range(5):
        folium.Marker(
            location=[result.elements()[i * 10].lat(), result.elements()[i * 10].lon()],
            popup='Some far ATM',
            icon=folium.Icon(color='red')
        ).add_to(m)

    # country border
    folium.GeoJson(
        afg_poly,
        style_function=lambda x: {'color': 'black'},
        name='Afghanistan'
    ).add_to(m)

    # country border
    folium.GeoJson(
        brazil_poly,
        style_function=lambda x: {'color': 'yellow'},
        name='Brazil'
    ).add_to(m)

    return m

Visualize ATMs (the nearest and few other) to understand whether everything is calculated correctly.

In [ ]:
visualize_ATM()

Finally let's calculate sum of distances from point in Afghanistan to the nearest ATMs.

In [ ]:
# in kilometers
from sklearn.metrics.pairwise import haversine_distances
from math import radians

EARTH_RADIUS = 6370
distances = [haversine_distances([[radians(result.elements()[i].lat()), radians(result.elements()[i].lon())],
                                  [radians(afg_point.y), radians(afg_point.x)]
                                  ]) for i in nearest_atm]
distances = np.array(distances).max(axis=(1, 2)) * EARTH_RADIUS
print('distances = ', list(np.round(distances, 2)))
print('final sum =', sum(distances))

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
url_NY_schools = 'https://data.cityofnewyork.us/api/views/a3nt-yts4/rows.csv?accessType=DOWNLOAD'
schools_raw = pd.read_csv(url_NY_schools)
schools_raw['geometry'] = schools_raw['the_geom'].apply(wkt.loads)
schools = gpd.GeoDataFrame(schools_raw, geometry='geometry')[['geometry']]
schools.head()

In [ ]:
schools = schools.geometry.values

Let us assume the starting point `park` is known. Now we search the point where Water Street and Washington Street cross together (`cross`).


In [ ]:
park = Point(40.7648980, -73.9725530)

water_ways = overpass.query('way["tiger:county"="Kings, NY"]["name"="Water Street"]; out body;')
washington_ways = overpass.query('way["tiger:county"="Kings, NY"]["name"="Washington Street"]; out body;')
print('Water Street ways found: ', water_ways.countElements())
print('Washington Street ways found: ', washington_ways.countElements())

In [ ]:
water_nodes = set()
for way in water_ways.elements():
    nodes = [(node.lat(), node.lon()) for node in way.nodes()]
    water_nodes |= set(nodes)

washington_nodes = set()
for way in washington_ways.elements():
    nodes = [(node.lat(), node.lon()) for node in way.nodes()]
    washington_nodes |= set(nodes)

cross = water_nodes & washington_nodes
cross = Point(cross.pop())

Luckily `cross` point is found. A rare case to derive something from geodata and get correct result!

Now we drop schools that are far enough from the route.

In [ ]:
park = Point(park.y, park.x)
cross = Point(cross.y, cross.x)
center = Point(0.5 * (park.x + cross.x), 0.5 * (park.y + cross.y))
R = center.distance(park)
borders = Polygon([Point(park.x + R, park.y + R), Point(park.x + R, cross.y - R),
                  Point(cross.x - R, cross.y - R), Point(cross.x - R, park.y + R)])
schools = [school for school in schools if borders.contains(school)]

In [ ]:
# function to create radius around school
def CreateBufferPolygon(point_in, resolution=10, radius=10):    
    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32632') # WGS84 UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 10 m buffer
    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84
    return poly_wgs

Here we construct vicinities for schools where a car should not pass.

In [ ]:
token = '5b3ce3597851110001cf62489a3c420f1b1745afaba2f280808a07e6'
clnt = client.Client(key=token)

map_params = {'location':([park.y, park.x]),
              'zoom_start': 12}
map1 = folium.Map(**map_params)

schools_poly = []
for school in tqdm(schools):
    folium.features.Marker(
        location=[school.y, school.x],
        popup='School'
    ).add_to(map1)

    # Create buffer polygons around construction sites with 10 m radius and low resolution
    school_coords = CreateBufferPolygon([school.x, school.y],
                                           resolution=10, # low resolution to keep polygons lean
                                           radius=100)
    schools_poly.append(school_coords)
    
    school_coords = [(y,x) for x,y in school_coords] # Reverse coords for folium/Leaflet
    folium.vector_layers.Polygon(locations=school_coords,
                                  color='crimson',
                                  fill_color='crimson',
                                  fill_opacity=0.2,
                                  weight=3).add_to(map1)
    
map1

Finally the routes are obtained using `openrouteservice`. You may see both the route that ignores schools on the way and the route that is aware of them.

In [ ]:
# style function
def style_function(color):
    return lambda feature: dict(color=color, weight=3, opacity=0.5)

def visualize_NY():
    # map
    m = folium.Map(
        location=[center.y, center.x],
        tiles='OpenStreetMap',
        zoom_start=12
    )

    # central park
    folium.Marker(
        location=[park.y, park.x],
        popup='Central Park',
        icon=folium.Icon()
    ).add_to(m)

    # final cross point
    folium.Marker(
        location=[cross.y, cross.x],
        popup='Water Street & Washington Street',
        icon=folium.Icon()
    ).add_to(m)

    # request normal route between locations (schools are ignored)
    request_params = {'coordinates': [[park.x, park.y],
                                      [cross.x, cross.y]],
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                    'instructions': 'false',}
    route_normal = clnt.directions(**request_params)
    folium.features.GeoJson(data=route_normal,
                            name='Route when ignoring schools',
                            style_function=style_function('#FF0000'),
                            overlay=True).add_to(m)

    # create buffer where the route is built
    route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.05)
    folium.features.GeoJson(data=geometry.mapping(route_buffer),
                            name='Route Buffer',
                            style_function=style_function('#FFFF00'),
                            overlay=True).add_to(m)

    # plot which schools fall into the buffer Polygon
    schools_buffer_poly = []
    for school_poly in schools_poly:
        poly = Polygon(school_poly)
        if route_buffer.intersects(poly):
            school_coords = [(y,x) for x,y in school_poly]
            folium.vector_layers.Polygon(locations=school_coords,
                                         overlay=True,
                                         color='crimson',
                                         fill_color='crimson',
                                         fill_opacity=0.2,
                                         weight=3).add_to(m)
            schools_buffer_poly.append(poly)
    
    # Add the site polygons to the request parameters
    request_params['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(schools_buffer_poly))}
    route_detour = clnt.directions(**request_params)

    folium.features.GeoJson(data=route_detour,
                            name='Route aware of schools',
                            style_function=style_function('#00FF00'),
                            overlay=True).add_to(m)

    m.add_child(folium.map.LayerControl())
    return m

In [ ]:
visualize_NY()